<a href="https://colab.research.google.com/github/Han555/ABCRental/blob/main/FloodDetectionModelWithMultipleInput.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import tensorflow as tf
import requests
import numpy as np
from tensorflow import keras
import keras.utils
import pandas as pd
from sklearn.utils import class_weight
from matplotlib import pyplot as plt
learning_rate=0.00001

model = keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/Flood Detection/vh_model.h5')
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
dataDirectory='/content/drive/MyDrive/Colab Notebooks/Flood Detection/VH Img/'
train_ds,val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataDirectory,    
    validation_split=0.3,
    seed=123456,
    subset='both',    
    color_mode='grayscale')

class_names = train_ds.class_names
print(class_names)
dataset_unbatched = tuple(train_ds.unbatch())
labels = []
for (image,label) in dataset_unbatched:
    labels.append(label.numpy())
labels = pd.Series(labels)

# adjustments
count = labels.value_counts().sort_index()
count.index = train_ds.class_names
print(labels)
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                 classes=np.unique(labels),
                                                 y=labels)

class_weights = dict(zip(np.unique(labels), class_weights))
print(class_weights)
history=model.fit(train_ds,
          batch_size=32,
          epochs=10,
          verbose=1,
          validation_data=val_ds,
          class_weight=class_weights)
pd.DataFrame(history.history).plot(figsize=(8,5))
plt.show()

imagePath='/content/drive/MyDrive/Colab Notebooks/Flood Detection/VH Img/True/True-VH-sen12floods_s1_labels_0137_2019_03_07.png'
test_image = keras.utils.load_img(imagePath,color_mode="grayscale") 
test_image = keras.utils.img_to_array(test_image)
input_arr = np.array([test_image])
#predict the result
result = model.predict(input_arr)
print(np.where(result > 0.5, 'True','False'))



Found 3327 files belonging to 2 classes.
Using 2329 files for training.
Using 998 files for validation.
['False', 'True']
0       0
1       1
2       0
3       0
4       1
       ..
2324    1
2325    0
2326    1
2327    0
2328    0
Length: 2329, dtype: int32
{0: 0.725093399750934, 1: 1.6106500691562933}
Epoch 1/10
73/73 [==============================] - 226s 3s/step - loss: 0.8313 - accuracy: 0.6587 - val_loss: 0.5955 - val_accuracy: 0.7505
Epoch 2/10
73/73 [==============================] - 146s 2s/step - loss: 0.6113 - accuracy: 0.6947 - val_loss: 0.5967 - val_accuracy: 0.6824
Epoch 3/10
 2/73 [..............................] - ETA: 1:58 - loss: 0.5687 - accuracy: 0.6719

KeyboardInterrupt: ignored

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.utils import Sequence
from keras.models import Sequential
from keras.layers import Input, Dense, concatenate
from keras.models import Model
from keras.utils import plot_model
import re
from sklearn.utils import class_weight
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import cv2


seed=123456

train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10
learning_rate=0.0001



df_dir='/content/drive/MyDrive/Colab Notebooks/Flood Detection/sen12floods_s1_labels_processed.csv'
df=pd.read_csv(df_dir)
df=df.head(50)


df["datetime"] = pd.to_datetime(df["datetime"],utc=True)

# Convert to Unix timestamp seconds
df["datetime"] = (df["datetime"] - pd.Timestamp("1970-01-01",tz='utc')) // pd.Timedelta("1s")

def process_attributes(train,val,test):
  categorical=['bbox_0','bbox_1','bbox_2','bbox_3']
  
  cs = MinMaxScaler(feature_range=(-1, 1))
  trainContinuous = cs.fit_transform(train["datetime"].values.reshape(-1, 1))
  valContinuous = cs.transform(val["datetime"].values.reshape(-1, 1))
  testContinuous = cs.transform(test["datetime"].values.reshape(-1, 1))

  trainCategorical = cs.fit_transform(train[categorical])
  valCategorical = cs.transform(val[categorical])
  testCategorical = cs.transform(test[categorical])

  trainX = np.hstack([trainCategorical, trainContinuous])
  valX = np.hstack([valCategorical, valContinuous])
  testX = np.hstack([testCategorical, testContinuous])
	# return the concatenated training and testing data
  return (trainX, valX,testX)

vh_images=[]
vv_images=[]
  
for i,r in df.iterrows():
  a = keras.utils.load_img(r.vh_img_href,color_mode="grayscale")
  a = a.crop((0,0,a.size[0],a.size[1]-10))
  a = a.resize((256,256))
  b = keras.utils.load_img(r.vv_img_href,color_mode="grayscale")
  b = b.crop((0,0,b.size[0],b.size[1]-10))
  b = b.resize((256,256))
  ar=keras.utils.img_to_array(a)
  ag = ar.astype('float32')
  ag /= 255

  br=keras.utils.img_to_array(b)
  bg = br.astype('float32')
  bg /= 255
  vh_images.append(ag)
  vv_images.append(bg)


split = train_test_split(df, vh_images, test_size=1 - train_ratio, random_state=42)
(trainAttrX, valAttrX, trainImagesVH, valImagesVH) = split
split = train_test_split(valAttrX, valImagesVH, test_size=test_ratio/(test_ratio + validation_ratio), random_state=43)
(valAttrX, testAttrX, valImagesVH, testImagesVH) = split

split = train_test_split(df, vv_images, test_size=0.25, random_state=42)
(trainAttrX, valAttrX, trainImagesVV, valImagesVV) = split
split = train_test_split(valAttrX, valImagesVV, test_size=test_ratio/(test_ratio + validation_ratio), random_state=43)
(valAttrX, testAttrX, valImagesVV, testImagesVV) = split


#print(trainAttrX)
trainY = trainAttrX["flood"]
valY = valAttrX["flood"]
testY = testAttrX["flood"]

class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                 classes=np.unique(trainY),
                                                 y=trainY)
model_weight=dict(zip(np.unique(trainY),class_weights))

labelencoder = LabelEncoder()
trainY = labelencoder.fit_transform(trainY) 
valY = labelencoder.transform(valY) 
testY = labelencoder.transform(testY) 

trainY = np.asarray(trainY).astype(np.float32)
valY = np.asarray(valY).astype(np.float32)
testY = np.asarray(testY).astype(np.float32)

trainAttrX,valAttrX,testAttrX=process_attributes(trainAttrX,valAttrX,testAttrX)

print(len(trainImagesVH))
print(len(trainImagesVV))
print(len(trainY))
 
# define our MLP network
def create_mlp(dim, regress=False):
	# define our MLP network
	model = Sequential()
	model.add(Dense(8, input_dim=dim, activation="relu",name="npl_dense_1"))
	model.add(Dense(4, activation="relu",name="npl_dense_2"))
	model.add(Dense(1, activation="sigmoid",name="npl_output_layer"))
	return model


vh_model = keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/Flood Detection/vh_model.h5')
vh_model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

vv_model = keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/Flood Detection/vv_model.h5')
vv_model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

mlp_model = create_mlp(trainAttrX.shape[1])
#Merging model A and B
merge_layer = concatenate([vh_model.output,vv_model.output,mlp_model.output],name="concatenated_layer")

#Final Layer
output_layer = Dense(1, activation = "sigmoid", name = "output_layer")(merge_layer)


#Model Definition 
merged = Model(inputs=[(vh_model.input,vv_model.input,mlp_model.input)],outputs=[output_layer], name = "merged_model")

#Model Details
merged.summary()
keras.utils.plot_model(merged, "merged_architecture.png", show_shapes=True)



merged.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
              metrics=['accuracy'])

vh_input_np=np.array(trainImagesVH,np.float32)
vv_input_np=np.array(trainImagesVV,np.float32)

vh_val_np=np.array(valImagesVH,np.float32)
vv_val_np=np.array(valImagesVV,np.float32)

vh_test_np=np.array(testImagesVH,np.float32)
vv_test_np=np.array(testImagesVV,np.float32)

history=merged.fit(x=[vh_input_np,vv_input_np,trainAttrX],
          y=trainY,
          batch_size=32,
          validation_batch_size=32,
          class_weight=model_weight,
          epochs=20,
          validation_data=([vh_val_np,vv_val_np,valAttrX],valY),
          verbose=1)
pd.DataFrame(history.history).plot(figsize=(8,5))
plt.show()

score = model.evaluate(([vh_test_np,vv_test_np,testAttrX],testY), verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


37
37
37
Model: "merged_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 conv2d_12_input (InputLayer)   [(None, None, None,  0           []                               
                                 1)]                                                              
                                                                                                  
 conv2d_8_input (InputLayer)    [(None, None, None,  0           []                               
                                 1)]                                                              
                                                                                                  
 conv2d_12 (Conv2D)             (None, None, None,   320         ['conv2d_12_input[0][0]']        
                                32)                                           